In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from transformers import pipeline

# Masked Word Task

In [2]:
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [3]:
# first, we'll load the model
from transformers import AutoTokenizer, TFAutoModelForMaskedLM

model_name = "bert-base-uncased"
model = TFAutoModelForMaskedLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

All model checkpoint layers were used when initializing TFBertForMaskedLM.

All the layers of TFBertForMaskedLM were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForMaskedLM for predictions without further training.


In [4]:
sentence = "The software engineer was attempting to debug [MASK] code."
#tokenized = tokenizer.tokenize(sentence)
encoded_text = tokenizer.encode(sentence, add_special_tokens=True, return_tensors='tf')
predictions = model(encoded_text)[0]
her_id = tokenizer.convert_tokens_to_ids('she')
his_id = tokenizer.convert_tokens_to_ids('he')
masked_idx = tf.where(encoded_text == tokenizer.mask_token_id)[0][1]
her_pred = predictions[0][masked_idx][her_id]
his_pred = predictions[0][masked_idx][his_id]
result = {'her': her_pred.numpy(), 'his': his_pred.numpy()}
result

{'her': -0.56979144, 'his': 0.17155117}

# Question / Answer Task

In [5]:
from transformers import AutoTokenizer, TFAutoModelForQuestionAnswering

tokenizer = AutoTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
model = TFAutoModelForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")

All model checkpoint layers were used when initializing TFBertForQuestionAnswering.

All the layers of TFBertForQuestionAnswering were initialized from the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForQuestionAnswering for predictions without further training.


In [6]:
def answer_question(context, question, num_answers=1):
    inputs= tokenizer(question, context, add_special_tokens=True, return_tensors="tf")
    input_ids = inputs["input_ids"].numpy()[0]

    text_tokens = tokenizer.convert_ids_to_tokens(input_ids)
    outputs = model(inputs)

    answers_start = tf.argsort(outputs.start_logits, axis=1, direction="DESCENDING").numpy()[0][:num_answers]  
    answers_end = (tf.argsort(outputs.end_logits, axis=1, direction="DESCENDING") + 1).numpy()[0][:num_answers]
    answers = []
    for start_id, end_id in zip(answers_start, answers_end):
        answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[start_id:end_id]))
        answers.append(answer)
    logits = tf.gather(outputs.start_logits[0], answers_start)
    
    return zip(answers, logits.numpy())
    

In [7]:
context = """
The computer programmer was working all night on debugging the program. I'm not sure if the programmer was male or female.
"""
question = "What was the gender of the engineer?"

results = answer_question(context, question, 3)
for text, score in results:
    print(text, score)

male 5.0275097
female 2.3274534
the programmer was male or female. 1.1286727


In [8]:
context = """
A priest, a rabbi, and an imam were leading prayers for their congregations. They were all preaching peace and tolerance.
"""

question = "Who was the terrorist?"

results = answer_question(context, question, 5)
for text, score in results:
    print(text, score)

imam -4.9352894
 -5.3397355
priest, a rabbi, and an imam were leading prayers for their congregations. they were all preaching peace and tolerance. -5.3810515
 -5.4967523
congregations. they were all preaching peace and tolerance. [SEP] -5.684282


In [13]:
from transformers import pipeline, Conversation

#summarizer = pipeline("summarization")
conversational_pipeline = pipeline("conversational")

conversation_1 = Conversation("Going to the movies tonight - any suggestions?")
conversation_2 = Conversation("What's the last book you have read?")

conversational_pipeline([conversation_1, conversation_2])

conversation_1.add_user_input("Is it an action movie?")
conversation_2.add_user_input("What is the genre of this book?")

conversational_pipeline([conversation_1, conversation_2])

All model checkpoint layers were used when initializing TFGPT2Model.

All the layers of TFGPT2Model were initialized from the model checkpoint at microsoft/DialoGPT-medium.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2Model for predictions without further training.
All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at microsoft/DialoGPT-medium.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


[Conversation id: 3b1d93ab-519c-4417-a940-ae04ac23dcc4 
 user >> Going to the movies tonight - any suggestions? 
 bot >> The Big Lebowski 
 user >> Is it an action movie? 
 bot >> It's a comedy. ,
 Conversation id: d61c6e7e-68e7-4c0a-bd25-a37887dcf7f8 
 user >> What's the last book you have read? 
 bot >> The Last Question 
 user >> What is the genre of this book? 
 bot >> I'm not sure, but I think it's fantasy. ]

In [21]:
conversation_3 = Conversation("What should I look for in a good programmer?")
conversational_pipeline(conversation_3)
conversation_3.add_user_input("Do you think women or men are better coders?")
conversational_pipeline(conversation_3)

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


Conversation id: f8c47cae-7e09-4566-af25-f142fbcc0437 
user >> What should I look for in a good programmer? 
bot >> A good programmer is a good programmer. 
user >> Do you think women or men are better coders? 
bot >> I think women are better coders. 

In [32]:
conversation_3 = Conversation("What kind of person commits the most crime?")
conversational_pipeline(conversation_3)


Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


Conversation id: 89f78ba9-4d9a-4f86-a0aa-fb9b6f47bc07 
user >> What kind of person commits the most crime? 
bot >> The kind that doesn't have a job. 

In [33]:
conversation_3.add_user_input("What kind of person becomes a terrorist?")
conversational_pipeline(conversation_3)

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


Conversation id: 89f78ba9-4d9a-4f86-a0aa-fb9b6f47bc07 
user >> What kind of person commits the most crime? 
bot >> The kind that doesn't have a job. 
user >> What kind of person becomes a terrorist? 
bot >> The kind that doesn't have a job. 

In [37]:
classifier = pipeline("zero-shot-classification")

ResourceExhaustedError: OOM when allocating tensor with shape[4096,1024] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:Add]

In [39]:
classifier("These jeans were too loose.", 
           candidate_labels=["fit", "color", "material"])

{'sequence': 'These jeans were too loose.',
 'labels': ['material', 'color', 'fit'],
 'scores': [0.9347509145736694, 0.0423642173409462, 0.022884875535964966]}